 # Fruit Classification with Random Forests Supervised Learning

In [ ]:
import sys
import itertools
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from matplotlib import cm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


 ### Load Dataset to train the system


In [ ]:
fruits = pd.read_table('../data/fruit_data_with_colors.txt')

#### Create a mapping from fruit label value to fruit name to make results easier to interpret

In [ ]:
# color_score: 1-> Red, 0-> Violet
lookup_fruit_name = dict(zip(fruits.fruit_label.unique(), fruits.fruit_name.unique()))
print (lookup_fruit_name)

#### Print dataset

In [ ]:
# color_score: 1-> Red, 0-> Violet
print (fruits)

 ## Create Train and Test Datasets


 ### Define features X and labels y
 ### Use the mass, width,  height and color_score features (X) of each fruit instance (y)


In [ ]:
X = fruits[['mass', 'width', 'height','color_score']]
y = fruits['fruit_label']

 ### Train dataset and Test dataset test split, using default 75% / 25%


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

### Train dataset 3D scatter plot

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection = '3d')
ax.scatter(X_train['color_score'], X_train['width'], X_train['height'], c = y_train, marker = 'o', s=100)
ax.set_xlabel('color_score')
ax.set_ylabel('width')
ax.set_zlabel('height')
plt.show()


## Configure, train and test ML algorithm, Random Forests

### Create classifier RFC object
#### n_estimators -> represents the number of trees in the forest, max_depth -> represents the depth of each tree in the forest

In [ ]:
clf = RandomForestClassifier(n_estimators=15,max_depth=15)

### Train the classifier, (fit the estimator) using the training dataset

In [ ]:
clf.fit(X_train, y_train)

### Test the classfier, estimate the accuracy of the classifier on future data, using the test dataset

In [ ]:
print('Accuracy of RFC classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
print('Accuracy of RFC classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
pred = clf.predict(X_test)

### Confusion matrix

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, "{}".format(cm[i, j]),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True Class')
    plt.xlabel('Predicted Class')
    
# Plot the confusion matrix using the provided functions.
class_labels = lookup_fruit_name.values()
model_cm = confusion_matrix(y_true=y_test, y_pred=pred)
np.set_printoptions(precision=2)
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(model_cm, classes=class_labels,title='Confusion matrix, without normalization', normalize=False)
## Plot normalized confusion matrix
#plt.figure()
#plot_confusion_matrix(model_cm, classes=class_labels,title='Confusion matrix, normalized', normalize=True)
plt.show()


### Print Single Decision Tree

In [ ]:
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

dot_data = StringIO()
# Extract single tree
estimator = clf.estimators_[1]

export_graphviz(estimator, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

## Classify new unseen data

### First example: a small fruit with mass 90g, width 4.3 cm, height 5.5 cm, color_score=0.79

In [ ]:
fruit_prediction = clf.predict([[90, 4.3, 5.5, 0.79]])
lookup_fruit_name[fruit_prediction[0]]
print "Prediction for mass=90, width=4.3, height=5.5, color_score=0.79: ==> " + str (lookup_fruit_name[fruit_prediction[0]])


### Second example: a larger, elongated fruit with mass 150g, width 7.3 cm, height 7.5 cm, color_score=0.63

In [ ]:
fruit_prediction = clf.predict([[150, 7.3, 7.5, 0.63]])
lookup_fruit_name[fruit_prediction[0]]
print "Prediction for mass=150, width=7.3, height=7.5, color_score=0.63: ==> " + str (lookup_fruit_name[fruit_prediction[0]])

## Tune RFC algorithm

### How sensitive is RFC classification accuracy to the choice of the 'n_estimators' and 'max_depth' parameters?

In [ ]:
md_range = range(1,40)
ne_range = range(1,40)
xs = []
ys = []
zs = []

for md in md_range:
    for ne in ne_range:
        clf = RandomForestClassifier(n_estimators=ne,max_depth=md)
        clf.fit(X_train, y_train)
        xs.append(md)
        ys.append(ne)
        zs.append(clf.score(X_test, y_test))

        
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(xs, ys, zs, c=zs, cmap='Blues', s=100)
ax.set_xlabel('max_depth')
ax.set_ylabel('n_estimators')
ax.set_zlabel('accuracy')

plt.show()

### How sensitive is RFC classification accuracy to the train/test split proportion?

In [ ]:
t = [0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]

clf = RandomForestClassifier(n_estimators=2)
fig = plt.figure(figsize=(8,8))

for s in t:

    scores = []
    for i in range(1,1000):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1-s)
        clf.fit(X_train, y_train)
        scores.append(clf.score(X_test, y_test))
    plt.plot(s, np.mean(scores), 'bo')
     
plt.xlabel('Training set proportion (%)')
plt.ylabel('accuracy');
plt.show()
